In [2]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
import os
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import sys
from pathlib import Path
import time
from pytorch3d.loss import chamfer_distance

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
import os
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import sys
from pathlib import Path
import time
from pytorch3d.loss import chamfer_distance

sys.path.append(str(Path.cwd().parent))

from Helpers.data import PointCloudDataset
import Helpers.PointCloudOpen3d as pc

if torch.cuda.is_available():
    device = "cuda"

elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f'Using: {device}')

Using: cuda


In [ ]:
point_size = 1024
train_dataset = PointCloudDataset("../Data/ModelNet40", point_size, 'train')
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
# test_dataset = PointCloudDataset("../Data/ModelNet40", point_size, 'test', object_classes=object_classes)
# test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True)
# print(len(test_dataset))
print(len(train_dataset))


2316


426


In [3]:
class PointCloudAE(nn.Module):
    def __init__(self, point_size, latent_size):
        super(PointCloudAE, self).__init__()
        
        self.latent_size = latent_size
        self.point_size = point_size
        
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, self.latent_size, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(self.latent_size)
        
        self.dec1 = nn.Linear(self.latent_size,256)
        self.dec2 = nn.Linear(256,256)
        self.dec3 = nn.Linear(256,self.point_size*3)

    def encoder(self, x): 
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, self.latent_size)
        return x
    
    def decoder(self, x):
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = self.dec3(x)
        return x.view(-1, self.point_size, 3)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [4]:

net = PointCloudAE(1024, 256).to(device)

optimizer = optim.AdamW(net.parameters(), lr=0.0001)

def train_epoch():
    epoch_loss = 0
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        
        x = data['points'].to(device)

        output = net(x.permute(0,2,1)) # transpose data for NumberxChannelxSize format

        loss = F.mse_loss(x, output)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    if i == 0:
        return epoch_loss / 1
    return epoch_loss/i


train_loss_list = []  
test_loss_list = []  

for i in range(10000) :

    startTime = time.time()
    
    train_loss = train_epoch() #train one epoch, get the average loss
    train_loss_list.append(train_loss)

    
    epoch_time = time.time() - startTime
    
    if i % 10 == 0:
        print(train_loss)

NameError: name 'train_loader' is not defined

In [27]:
torch.save(net.state_dict(), 'trained_models/furniture_10k')

In [49]:
torch.save(net, 'trained_models/furniture_model_weights_50k')

In [50]:
model = torch.load('trained_models/furniture_model_weights_50k', weights_only = False)

In [37]:
model = net
x = next(iter(train_loader))['points'][0]

cloud = pc.get_point_cloud(x)
pc.visualize_point_cloud(cloud)

with torch.no_grad():
    data = x.unsqueeze(0).permute(0,2,1).to(device)

    rec_x = np.array(model(data)[0].to('cpu'))
    cloud = pc.get_point_cloud(rec_x)
    pc.visualize_point_cloud(cloud)
# x = next(iter(train_loader))['points'][0]

# cloud = pc.get_point_cloud(x)
# pc.visualize_point_cloud(cloud)

# with torch.no_grad():
#     data = x.unsqueeze(0).permute(0,2,1).to(device)

#     rec_x = np.array(model(data)[0].to('cpu'))
#     cloud = pc.get_point_cloud(rec_x)
#     pc.visualize_point_cloud(cloud)

In [52]:
model = net
x = next(iter(test_loader))['points'][0]

cloud = pc.get_point_cloud(x)
pc.visualize_point_cloud(cloud)

with torch.no_grad():
    data = x.unsqueeze(0).permute(0,2,1).to(device)

    rec_x = np.array(model(data)[0].to('cpu'))
    cloud = pc.get_point_cloud(rec_x)
    pc.visualize_point_cloud(cloud)